In [107]:
import argparse, json, copy, os
from six.moves import cPickle as pickle

from deep_dialog.dialog_system import DialogManager, text_to_dict
from deep_dialog.agents import AgentCmd, InformAgent, RequestAllAgent, RandomAgent, EchoAgent, RequestBasicsAgent, AgentDQN
from deep_dialog.usersims import RuleSimulator

from deep_dialog import dialog_config
from deep_dialog.dialog_config import *

from deep_dialog.nlu import nlu
from deep_dialog.nlg import nlg
from deep_dialog.nlg.lstm_decoder_tanh import lstm_decoder_tanh
from IPython import *
from notebook_util import *
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## load_nlg_model

In [31]:
nlg_model_path = './deep_dialog/models/nlg/lstm_tanh_relu_[1468202263.38]_2_0.610.p'
nlg_model = nlg()
nlg_model.load_nlg_model(nlg_model_path)

### detail

In [99]:
model_params = pickle.load(open(nlg_model_path, 'rb'), encoding='latin1')
display.HTML('<h4>{}</h4> '.format("model_params Keys:"))

In [100]:
list(model_params.keys())

['act_dict',
 'template_word_dict',
 'epochs',
 'word_dict',
 'params',
 'slot_dict',
 'model',
 'slot_val_dict']

#### acttions

In [56]:
model_params['act_dict']

{'closing': 5,
 'confirm_answer': 3,
 'confirm_question': 2,
 'deny': 9,
 'greeting': 4,
 'inform': 1,
 'multiple_choice': 6,
 'not_sure': 10,
 'request': 0,
 'thanks': 7,
 'welcome': 8}

#### word dictionary

In [123]:
word_dict=model_params['word_dict']
keys = word_dict.keys()
print("word dict: keys = {}".format(len(keys)))
print("Sample of word dict:")
random_keys = np.random.choice(list(keys), 6)
print([(k, word_dict[k]) for k in random_keys])

word dict: keys = 1527
Sample of word dict:
[('21', 829), ('ness', 1149), ('97232', 458), ('7:55pm', 1392), ('lansing', 1120), ('come', 536)]


In [129]:
template_word_dict = model_params['template_word_dict']
keys = template_word_dict.keys()
print("template word dict: keys = {}".format(len(keys)))
print("Sample of word dict:")
random_keys = np.random.choice(list(keys), 5)
print([(k, template_word_dict[k]) for k in random_keys])

template word dict: keys = 1047
Sample of word dict:
[('shall', 508), ("i'm", 95), ('across', 557), ('7038', 414), ('gotten', 265)]


In [130]:
model_params['epochs']

200

#### weigths

In [45]:
model_params['model']['WLSTM'].shape

(1148, 400)

In [47]:
model_params['model']['Wah'].shape

(1116, 400)

In [49]:
model_params['model']['Wd'].shape

(100, 1047)

In [53]:
model_params['model']['bah'].shape

(1, 400)

In [54]:
model_params['model']['bd'].shape

(1, 1047)

#### model dimension

In [15]:
hidden_size = model_params['model']['Wd'].shape[0]
print("hidden_size: {}".format(hidden_size))

hidden_size: 100


In [16]:
output_size = model_params['model']['Wd'].shape[1]
print("output_size: {}".format(output_size))

output_size: 1047


In [27]:
if model_params['params']['model'] == 'lstm_tanh': # lstm_tanh
        print("LSTM_Tanh")
        diaact_input_size = model_params['model']['Wah'].shape[0]
        input_size = model_params['model']['WLSTM'].shape[0] - hidden_size - 1
        rnnmodel = lstm_decoder_tanh(diaact_input_size, input_size, hidden_size, output_size)
        print("diaact_input_size = {}".format(diaact_input_size))
        print("input_size = {}".format(input_size))

LSTM_Tanh
diaact_input_size = 1116
input_size = 1047
